In [ ]:
import sys
!{sys.executable} -m pip install python-docx
!{sys.executable} -m pip install --upgrade transformers[torch] datasets evaluate sacremoses
!{sys.executable} -m pip install --upgrade mlxtend wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00
  Attempting uninsta

In [ ]:
import docx
import os, re
from collections import Counter

# Data loading code

Counter(labels)


Counter({0: 168, 1: 195})

In [ ]:

from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(labels, labels)
dummy_clf.score(labels, labels)

0.5371900826446281

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("setu4993/LaBSE", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at setu4993/LaBSE and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(501153, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
summary(model)

Layer (type:depth-idx)                                  Param #
BertForSequenceClassification                           --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              384,885,504
│    │    └─Embedding: 3-2                              393,216
│    │    └─Embedding: 3-3                              1,536
│    │    └─LayerNorm: 3-4                              1,536
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             85,054,464
│    └─BertPooler: 2-3                                  --
│    │    └─Linear: 3-7                                 590,592
│    │    └─Tanh: 3-8                                   --
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           1,538
Total params: 4

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
import numpy as np
import wandb, gc, os, pickle, datasets, transformers, torch, evaluate

from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from datasets import concatenate_datasets, Dataset
from transformers import (
    BertConfig,
    AutoTokenizer,
    BertModel,
    BertTokenizerFast,
    BertForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments)

transformers.logging.set_verbosity_error()

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def preprocess_function(examples, tokenizer):
    return tokenizer.batch_encode_plus(examples["text"], is_split_into_words=False, truncation=True, max_length=512, padding='max_length', pad_to_max_length=True)

model_name = "setu4993/LaBSE"
runtime_name = model_name.replace('/','_')

tokenizer = AutoTokenizer.from_pretrained(model_name, padding="max_length", max_length=512, truncation=True, do_lower_case=False)


dataset_dict = {'label': labels, 'text': texts}
dataset_hf = Dataset.from_dict(dataset_dict)
dataset_hf = dataset_hf.map(lambda p: preprocess_function(p, tokenizer), batched=True)

labels = np.array(labels)
texts = np.array(texts)

true_labels = [] # ground truth
predictions = [] # model predictions

batch_size = 6
fold_nr=0
all_results = []
device = "cuda"

kf = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kf.split(texts):

  print(f'Fold_nr= {fold_nr}') ; fold_nr += 1

  model_config = BertConfig.from_pretrained(model_name, num_labels=2)
  model = BertForSequenceClassification.from_pretrained(model_name, config=model_config)

  training_args = TrainingArguments(
      do_eval=True,
      output_dir='./results-argmining',          # output directory
      overwrite_output_dir=True,
      num_train_epochs=10,              # total number of training epochs
      per_device_train_batch_size=batch_size,   # 8 @A100 batch size per device during training
      per_device_eval_batch_size=batch_size,    # 8 @A100 batch size for evaluation
      warmup_steps=80,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=80,
      evaluation_strategy="steps",     # Evaluation is done at the end of each epoch, alternatives: "steps", "epoch"
      save_strategy="steps",           # Save is done at the end of each epoch, alternatives: "steps", "epoch"
      # save_steps=500                 # Number of updates steps before two checkpoint saves if save_strategy="steps",
      learning_rate=2e-5,
      eval_steps = 25,
      save_total_limit = 1,
      load_best_model_at_end=True,     # Whether or not to load the best model found during training at the end of training.
      metric_for_best_model='accuracy',
      disable_tqdm=False,
      run_name = runtime_name
  )

  train_dataset = dataset_hf.select(train_index)
  test_dataset = dataset_hf.select(test_index)

  # let's have validation set from 5% of train:
  train_dataset = train_dataset.train_test_split(test_size=0.05, shuffle=True)

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length', max_length=512, return_tensors='pt')

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset['train'],  # train part of train
      eval_dataset=train_dataset['test'],  # this is validation
      data_collator=data_collator,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )
  trainer.train()

  predictions_obj = trainer.predict(test_dataset)
  y_pred = np.argmax(predictions_obj.predictions, axis=1)
  test_dataset = test_dataset.add_column("y_pred", y_pred)

  predictions.append( y_pred )
  true_labels.append( labels[test_index] )

print( classification_report(np.concatenate(true_labels).ravel().tolist(), np.concatenate(predictions).ravel().tolist() ) )


tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Map:   0%|          | 0/363 [00:00<?, ? examples/s]

...

Fold_nr= 9


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
25,No log,0.736702,0.470588,0.526316,0.416667,0.714286
50,No log,0.614597,0.647059,0.400000,0.666667,0.285714
75,No log,0.569739,0.647059,0.500000,0.600000,0.428571
100,0.616800,0.928179,0.647059,0.571429,0.571429,0.571429
125,0.616800,1.290169,0.588235,0.461538,0.500000,0.428571
150,0.616800,1.604055,0.588235,0.461538,0.500000,0.428571
175,0.333400,1.878414,0.588235,0.461538,0.500000,0.428571
200,0.333400,1.650781,0.588235,0.533333,0.500000,0.571429
225,0.333400,2.061476,0.647059,0.571429,0.571429,0.571429
250,0.115800,2.518717,0.588235,0.461538,0.500000,0.428571


Flattening the indices:   0%|          | 0/36 [00:00<?, ? examples/s]

eval/accuracy,▁███▆▆▆▆█▆▆▆▆▃▃▆▆▃▃▆
eval/f1,▆▁▅█▄▄▄▆█▄▄▄▄▂▂▄▄▂▂▄
eval/loss,▁▁▁▂▃▄▅▄▅▇▇▇█▇██████
eval/precision,▁█▆▅▃▃▃▃▅▃▃▃▃▁▁▃▃▁▁▃
eval/recall,█▁▃▆▃▃▃▆▆▃▃▃▃▃▃▃▃▃▃▃
eval/runtime,▇▇▇█▇█▇██▇▇▇▇▇█▁▇▇██
eval/samples_per_second,▂▂▂▁▂▁▂▁▁▂▂▁▂▂▁█▁▂▁▁
eval/steps_per_second,▂▂▂▁▂▁▂▁▁▂▂▁▂▂▁█▁▂▁▁
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
train/learning_rate,█▇▅▄▂▁


              precision    recall  f1-score   support

           0       0.83      0.86      0.85       168
           1       0.88      0.85      0.86       195

    accuracy                           0.86       363
   macro avg       0.86      0.86      0.86       363
weighted avg       0.86      0.86      0.86       363

